In [1]:
from typing import List
import torch
import torch.nn as nn


class ForLoop(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, xs: List[torch.Tensor]):
        while xs[0].numel() > 0:
            xs[0] = torch.zeros((0, 0))
        return xs


for_loop_model = ForLoop()
s_for_loop_model = torch.jit.script(for_loop_model)
print(s_for_loop_model.graph)


graph(%self : __torch__.ForLoop,
      %xs.1 : Tensor[]):
  %13 : NoneType = prim::Constant()
  %2 : int = prim::Constant[value=9223372036854775807]() # /tmp/ipykernel_485535/2648660742.py:11:8
  %4 : int = prim::Constant[value=0]() # /tmp/ipykernel_485535/2648660742.py:11:17
  %24 : Tensor = aten::__getitem__(%xs.1, %4) # /tmp/ipykernel_485535/2648660742.py:11:14
  %25 : int = aten::numel(%24) # /tmp/ipykernel_485535/2648660742.py:11:14
  %26 : bool = aten::gt(%25, %4) # /tmp/ipykernel_485535/2648660742.py:11:14
   = prim::Loop(%2, %26) # /tmp/ipykernel_485535/2648660742.py:11:8
    block0(%9 : int):
      %12 : int[] = prim::ListConstruct(%4, %4)
      %17 : Tensor = aten::zeros(%12, %13, %13, %13, %13) # /tmp/ipykernel_485535/2648660742.py:12:20
      %19 : Tensor[] = aten::_set_item(%xs.1, %4, %17) # /tmp/ipykernel_485535/2648660742.py:12:12
      %5 : Tensor = aten::__getitem__(%xs.1, %4) # /tmp/ipykernel_485535/2648660742.py:11:14
      %6 : int = aten::numel(%5) # /tmp/ipykernel

In [2]:
from typing import List
import torch
import torch.nn as nn
from brt.routers.rand import RandomScatterRouter, RandomGatherRouter
from brt.primitive.helper import symbolize


class ForLoop(nn.Module):
    def __init__(self):
        super().__init__()
        self.scatter = RandomScatterRouter(dst_num=2)
        self.moe_decoder = nn.Linear(10, 10)
        self.gather = RandomGatherRouter(dst_num=2)

    def forward(self, x: torch.Tensor):
        target = torch.zeros((1, 1), device=x.device)
        while target.numel() > 0:
            route_results = self.scatter(x)
            target = route_results[0]
            x = self.moe_decoder(route_results[0])
            x = self.gather([x, route_results[1]])
        return x


for_loop_model = ForLoop()

x = torch.randn(1, 10)
y = for_loop_model(x)
print(y)
s_for_loop_model = torch.jit.script(symbolize(for_loop_model))
print(s_for_loop_model.graph)


tensor([[-0.2335,  0.2240,  0.8765,  0.7727,  0.4278, -0.7833, -1.2467,  0.7585,
         -0.7476, -0.7688]], grad_fn=<AliasBackward0>)
graph(%self : __torch__.___torch_mangle_7.ForLoop,
      %x.1 : Tensor):
  %8 : NoneType = prim::Constant()
  %2 : int = prim::Constant[value=1]() # /tmp/ipykernel_1381397/910957184.py:16:30
  %12 : int = prim::Constant[value=9223372036854775807]() # /tmp/ipykernel_1381397/910957184.py:17:8
  %15 : int = prim::Constant[value=0]() # /tmp/ipykernel_1381397/910957184.py:17:31
  %5 : Device = prim::device(%x.1)
  %7 : int[] = prim::ListConstruct(%2, %2)
  %target.1 : Tensor = aten::zeros(%7, %8, %8, %5, %8) # /tmp/ipykernel_1381397/910957184.py:16:17
  %49 : int = aten::numel(%target.1) # /tmp/ipykernel_1381397/910957184.py:17:14
  %50 : bool = aten::gt(%49, %15) # /tmp/ipykernel_1381397/910957184.py:17:14
  %x : Tensor = prim::Loop(%12, %50, %x.1) # /tmp/ipykernel_1381397/910957184.py:17:8
    block0(%18 : int, %x.19 : Tensor):
      %scatter : __torch__.